# 计算每天的情感倾向,作为边的权重
# 每个人可能有很多条评论,结果相加

In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm
from snownlp import SnowNLP

file_names = os.listdir(os.getcwd())  # 路径
comment_file = (re.findall('user_L[0-9]+_post_comment.csv', ''.join(file_names)))
print(len(comment_file))
data=pd.DataFrame()
for file in comment_file:
    data =data.append(pd.read_csv(file, encoding='utf_8_sig'))

data['content'].replace(to_replace=' ',value='',inplace=True)
data.dropna(subset=['content'],inplace=True)
data['sentiment']=None
data['score']=None

11


In [2]:
# 去除自己的回复自己的
# data[data['author_user_name']==data['reply1user']]
data.drop(data[data['author_user_name']==data['reply2user']].index,inplace=True)
print(data[data['author_user_name']==data['reply2user']])

Empty DataFrame
Columns: [author_user_name, post_id, reply2user, content, date, sentiment, score]
Index: []


In [3]:
for index,row in tqdm(data.iterrows()):
    data.at[index,'score']=SnowNLP(row['content']).sentiments
    # break

20756it [01:43, 201.49it/s]


In [4]:
data.loc[data['score']>=0.7,'sentiment']=1
data.loc[data[(0.3<data['score'])&(data['score']<0.7)].index,'sentiment']=0
data.loc[data['score']<=0.3,'sentiment']=-1

# 计算周围节点 对该节点的评价求和
# 计算每个节点 对该节点的评价

In [5]:
node=pd.read_csv(f"node.csv", encoding='utf_8_sig')
edge=pd.read_csv(f"edge.csv", encoding='utf_8_sig')
node['sentiment_sum']=0
node['sentiment_category']=0 # 取绝对值然后分类,为了方便可视化
edge['sentiment_classify']=0 # 求和之后 进行分类 -1 0
for name,group in tqdm(data.groupby('reply2user')):
    # print(name,group['sentiment'].sum())
    index= node['author_user_name']==name
    sentiment_sum=group['sentiment'].sum()
    if sentiment_sum>0:
        node.at[index,'sentiment_category']=1
    elif sentiment_sum<0:
        node.at[index,'sentiment_category']=-1
    else:
        node.at[index,'sentiment_category']=0
    node.at[index,'sentiment_sum']=abs(sentiment_sum)

    for n,g in group.groupby('author_user_name'):
        edge.loc[edge[(edge['author_user_name']==str(n))&(edge['reply2user']==str(name))].index,'sentiment_classify']=g['sentiment'].sum()
    # break
edge.loc[edge['sentiment_classify']>0,'sentiment_classify']=1
edge.loc[edge['sentiment_classify']==0,'sentiment_classify']=0
edge.loc[edge['sentiment_classify']<0,'sentiment_classify']=-1
print(node['sentiment_sum'].describe())
print(edge['sentiment_classify'].describe())

100%|██████████| 342/342 [01:07<00:00,  5.10it/s]


count    16185.000000
mean         0.177263
std          9.080911
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       1128.000000
Name: sentiment_sum, dtype: float64
count    18276.000000
mean         0.027577
std          0.724558
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: sentiment_classify, dtype: float64


# 将观点求总

In [6]:
from ast import literal_eval
post_file = (re.findall('user_L[0-9]+_post_filter.csv', ''.join(file_names)))
print(len(post_file))
post=pd.DataFrame()
for file in post_file:
    post =post.append(pd.read_csv(file, encoding='utf_8_sig'))

node['prop_adj_list']=None
for name,group in tqdm(post.groupby('author_user_name')):
    tmp=[]
    for index,row in group.iterrows():
        tmp.extend(literal_eval(row['prop_adj_list']))
    tmp=list(set(tmp))
    node.at[node[node['author_user_name']==str(name)].index[0],'prop_adj_list']=tmp
    # print(name,group['sentiment'].sum())


100%|██████████| 423/423 [00:00<00:00, 449.78it/s]


11


In [7]:
# save
node.to_csv('node.csv',index=False,encoding='utf_8_sig')
edge.to_csv('edge.csv',index=False,encoding='utf_8_sig')


